### Convert 3D volumetric data to 2D images

In [1]:
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import h5py
import helpers as H
from tqdm import tqdm

#Root directory of the project
ROOT_DIR = os.path.abspath(".")
TRAIN_DATASET_PATH = os.path.join(ROOT_DIR, "dataset")
#train_data_paths = glob.glob(os.path.join(TRAIN_DATASET_PATH,'img_datas_1','*.npy'))
train_data_paths = os.listdir(os.path.join(TRAIN_DATASET_PATH, "img_datas_1"))
print(len(train_data_paths))

218


In [2]:
def to_3D(arr, resize_dim=None):
    img = np.asarray(np.dstack((arr, arr, arr)), dtype=np.float64)
    if resize_dim is not None:
        img=cv2.resize(img,(resize_dim, resize_dim),interpolation=cv2.INTER_AREA)    
    else:
        pass
    return img

In [3]:
def get_number(filename):
    return int(filename[:filename.find('.')])

def sort_paths(paths):
    paths.sort(key = get_number)
    return paths

In [4]:
train_data_paths[:4]

['1.npy', '10.npy', '100.npy', '101.npy']

In [5]:
sort_paths(train_data_paths)[:5]

['1.npy', '2.npy', '3.npy', '4.npy', '5.npy']

In [6]:
y_train = np.load("y_train_clef.npy")
y_train.shape

(218, 2)

In [7]:
y_train_flat = np.array([np.argmax(x) for x in y_train])
y_train_flat.shape

(218,)

In [8]:
y_train_flat

array([0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
      dtype=int64)

In [14]:
low_paths = os.path.join(TRAIN_DATASET_PATH, "2D_images/0")
high_paths = os.path.join(TRAIN_DATASET_PATH, "2D_images/1")
low_paths, high_paths

('E:\\ECIR_2020\\ecir2020\\2D\\dataset\\2D_images/0',
 'E:\\ECIR_2020\\ecir2020\\2D\\dataset\\2D_images/1')

In [15]:
H.create_directory(os.path.join(low_paths))
H.create_directory(os.path.join(high_paths))

In [16]:
x_train_2D = []
y_train_2D = []

img_number = 1
for array_path, label in zip(sort_paths(train_data_paths)[:197], y_train_flat):
    img = np.load("{}/img_datas_1/{}".format(TRAIN_DATASET_PATH, array_path))
    print(array_path, "-----", img.shape, "---", label)
    
    slice_number = img.shape[-1]
    if label == 0:
        for slc in range(slice_number):
            
            # Save as numpy array
            #x_train_2D.append(img[:,:,slc])
            #y_train_2D.append(0)
            
            # 0 folder
            np.save("{}/{}_{}.npy".format(low_paths, img_number, slc), img[:,:,slc])
            
    else:
        for slc in range(slice_number):
            
            #x_train_2D.append(img[:,:,slc])
            #y_train_2D.append(1)
            
            # 1 folder
            np.save("{}/{}_{}.npy".format(high_paths, img_number, slc), img[:,:,slc])
            
    img_number+=1

1.npy ----- (512, 512, 128) --- 0
2.npy ----- (512, 512, 122) --- 0
3.npy ----- (512, 512, 113) --- 0
4.npy ----- (512, 512, 120) --- 0
5.npy ----- (512, 512, 116) --- 1
6.npy ----- (512, 512, 131) --- 0
7.npy ----- (512, 512, 123) --- 0
8.npy ----- (512, 512, 132) --- 1
9.npy ----- (512, 512, 144) --- 1
10.npy ----- (512, 512, 131) --- 0
11.npy ----- (512, 512, 133) --- 0
12.npy ----- (512, 512, 132) --- 1
13.npy ----- (512, 512, 129) --- 1
14.npy ----- (512, 512, 116) --- 0
15.npy ----- (512, 512, 130) --- 0
16.npy ----- (512, 512, 125) --- 1
17.npy ----- (512, 512, 104) --- 1
18.npy ----- (512, 512, 114) --- 0
19.npy ----- (512, 512, 132) --- 0
20.npy ----- (512, 512, 112) --- 0
21.npy ----- (512, 512, 112) --- 0
22.npy ----- (512, 512, 133) --- 0
23.npy ----- (512, 512, 104) --- 1
24.npy ----- (512, 512, 122) --- 0
25.npy ----- (512, 512, 129) --- 0
26.npy ----- (512, 512, 100) --- 1
27.npy ----- (512, 512, 131) --- 0
28.npy ----- (512, 512, 129) --- 1
29.npy ----- (512, 512, 102) 

In [12]:
len(os.listdir(low_paths)), len(os.listdir(high_paths))

(868, 392)

In [1]:
#x_train_2D = np.array(x_train_2D)
#y_train_2D = np.array(y_train_2D)
#x_train_2D.shape, y_train_2D.shape

In [22]:
#a = np.load("{}/img_datas_1/{}".format(TRAIN_DATASET_PATH, "3.npy"))
#a.shape

In [21]:
#x = a[:,:,50]
#plt.imshow(x, cmap='gray')

In [29]:
np.count_nonzero(y_train_2D == 1), np.count_nonzero(y_train_2D == 0)

(392, 868)